In [179]:
import numpy as np
import pandas as pd
import datetime
import math

In [189]:
dam = pd.read_csv('dam_out.csv')
dam

,year,month,day,date,market,hour,price,volume,day_of_week,value,price_min,price_max,price_mean,price_morning_max,price_evening_max,cat
0,2016,1,1,2016-01-01,1,0,40.10,477.5,4,19147.750,35.09,60.08,45.98250,48.03,60.08,off
1,2016,1,1,2016-01-01,1,1,40.08,486.6,4,19502.928,35.09,60.08,45.98250,48.03,60.08,off
2,2016,1,1,2016-01-01,1,2,40.06,495.6,4,19853.736,35.09,60.08,45.98250,48.03,60.08,off
3,2016,1,1,2016-01-01,1,3,40.08,496.6,4,19903.728,35.09,60.08,45.98250,48.03,60.08,off
4,2016,1,1,2016-01-01,1,4,40.07,502.9,4,20151.203,35.09,60.08,45.98250,48.03,60.08,off
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46723,2021,4,29,2021-04-29,1,4,39.97,126.1,3,5040.217,39.79,296.82,98.74250,195.01,296.82,off
46724,2021,4,29,2021-04-29,1,12,68.06,71.0,3,4832.260,39.79,296.82,98.74250,195.01,296.82,standard
46725,2021,4,30,2021-04-30,1,0,44.04,59.6,4,2624.784,38.97,295.98,98.21875,190.00,295.98,off
46726,2021,4,30,2021-04-30,1,8,190.00,72.0,4,13680.000,38.97,295.98,98.21875,190.00,295.98,morning


In [183]:
price = pd.read_csv('dam.csv')

price

,hourlyid,market,tradedate,hour,price,volume
0,26345,1,2016-01-01,0,40.10,477.5
1,26346,1,2016-01-01,1,40.08,486.6
2,26347,1,2016-01-01,2,40.06,495.6
3,26348,1,2016-01-01,3,40.08,496.6
4,26349,1,2016-01-01,4,40.07,502.9
...,...,...,...,...,...,...
46723,73029,1,2021-04-29,4,39.97,126.1
46724,73037,1,2021-04-29,12,68.06,71.0
46725,73049,1,2021-04-30,0,44.04,59.6
46726,73057,1,2021-04-30,8,190.00,72.0


In [35]:
flow = pd.read_csv('flow.csv')

flow['date']=pd.to_datetime(flow['flowdate'],format="%Y-%m-%d")#"%d/%m/%Y")
flow=flow.drop(['flowdate','flowid','market'],axis=1)

flow=flow.set_index([pd.DatetimeIndex(flow['date'])])
flow['year']=flow.index.year
flow['month']=flow.index.month
flow['day']=flow.index.day
flow['day_of_week']=flow.index.dayofweek
flow['month_no']=flow['year']+flow['month']/100

flow['hour_ref'] = flow.apply(lambda x: ('weekend_' + str(x['hour']) if x['day_of_week']>4 else 'weekday_' + str(x['hour'])),axis=1)
flow=flow.set_index([pd.DatetimeIndex(flow['date']),'hour'])
flow['price']=price['price']
flow['value']=flow['price']*flow['flow']
flow=flow.drop(['date'],axis=1)
flow=flow.reset_index()
flow

,date,hour,flow,from,to,year,month,day,day_of_week,month_no,hour_ref,price,value
0,2016-01-02,13,50,BOT,ZIMA,2016,1,2,5,2016.01,weekend_13,50.05,2502.50
1,2016-01-02,14,63,ZIMA,ZIM,2016,1,2,5,2016.01,weekend_14,50.31,3169.53
2,2016-01-02,15,263,ZIM,ZAMZ,2016,1,2,5,2016.01,weekend_15,50.49,13278.87
3,2016-01-02,16,100,MOZN_EDM,ZIMA,2016,1,2,5,2016.01,weekend_16,56.07,5607.00
4,2016-01-02,17,120,ZIM,ZAMZ,2016,1,2,5,2016.01,weekend_17,56.06,6727.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
333320,2021-03-31,19,3,RSAN,RSAS,2021,3,31,2,2021.03,weekday_19,148.01,444.03
333321,2021-03-31,20,13,RSAN,BOT,2021,3,31,2,2021.03,weekday_20,139.06,1807.78
333322,2021-03-31,21,33,MOZS,RSAN,2021,3,31,2,2021.03,weekday_21,89.03,2937.99
333323,2021-03-31,22,43,ZIMA,ZIM,2021,3,31,2,2021.03,weekday_22,36.98,1590.14


In [36]:
f_to=flow[['date','hour','flow','value','to','year','month','day','day_of_week','month_no','hour_ref']].rename(columns={'to': 'node'})
f_from=flow[['date','hour','flow','value','from','year','month','day','day_of_week','month_no','hour_ref']].rename(columns={'from': 'node'})
f_from['flow']=-f_from['flow']
f_from['value']=-f_from['value']

flows=pd.concat([f_to, f_from])

flows

,date,hour,flow,value,node,year,month,day,day_of_week,month_no,hour_ref
0,2016-01-02,13,50,2502.50,ZIMA,2016,1,2,5,2016.01,weekend_13
1,2016-01-02,14,63,3169.53,ZIM,2016,1,2,5,2016.01,weekend_14
2,2016-01-02,15,263,13278.87,ZAMZ,2016,1,2,5,2016.01,weekend_15
3,2016-01-02,16,100,5607.00,ZIMA,2016,1,2,5,2016.01,weekend_16
4,2016-01-02,17,120,6727.20,ZAMZ,2016,1,2,5,2016.01,weekend_17
...,...,...,...,...,...,...,...,...,...,...,...
333320,2021-03-31,19,-3,-444.03,RSAN,2021,3,31,2,2021.03,weekday_19
333321,2021-03-31,20,-13,-1807.78,RSAN,2021,3,31,2,2021.03,weekday_20
333322,2021-03-31,21,-33,-2937.99,MOZS,2021,3,31,2,2021.03,weekday_21
333323,2021-03-31,22,-43,-1590.14,ZIMA,2021,3,31,2,2021.03,weekday_22


In [37]:
flows.loc[flows['node']=='ZIMA','node']='ZIM'
flows.loc[flows['node']=='MOZN_EDM','node']='MOZ'
flows.loc[flows['node']=='MOZN_HCB','node']='MOZ'
flows.loc[flows['node']=='MOZS','node']='MOZ'
flows.loc[flows['node']=='RSAS','node']='RSA'
flows.loc[flows['node']=='RSAN','node']='RSA'

In [38]:
flows=flows.reset_index().groupby(['date','hour','node','year','month','day','day_of_week','month_no','hour_ref']).sum().reset_index()
flows=flows.loc[flows['flow']!=0]
flows

,date,hour,node,year,month,day,day_of_week,month_no,hour_ref,index,flow,value
0,2016-01-01,0,ZAMC,2016,1,1,4,2016.01,weekday_0,5607,160,6416.00
1,2016-01-01,0,ZAMZ,2016,1,1,4,2016.01,weekday_0,11320,10,401.00
2,2016-01-01,0,ZIM,2016,1,1,4,2016.01,weekday_0,5713,-170,-6817.00
3,2016-01-01,1,ZAMC,2016,1,1,4,2016.01,weekday_1,5714,160,6412.80
4,2016-01-01,1,ZAMZ,2016,1,1,4,2016.01,weekday_1,11322,10,400.80
...,...,...,...,...,...,...,...,...,...,...,...,...
289677,2021-03-31,22,ZAMC,2021,3,31,2,2021.03,weekday_22,333292,43,1590.14
289680,2021-03-31,23,BOT,2021,3,31,2,2021.03,weekday_23,666156,81,2993.76
289681,2021-03-31,23,MOZ,2021,3,31,2,2021.03,weekday_23,666292,-164,-6061.44
289682,2021-03-31,23,NAM,2021,3,31,2,2021.03,weekday_23,333309,42,1552.32


In [190]:
flows=flows.set_index(['year','month','day','hour'])
dam=dam.set_index(['year','month','day','hour'])

In [193]:
flows['cat']=dam['cat']
flows.reset_index()

,year,month,day,hour,date,node,day_of_week,month_no,hour_ref,index,flow,value,cat
0,2016,1,1,0,2016-01-01,ZAMC,4,2016.01,weekday_0,5607,160,6416.00,off
1,2016,1,1,0,2016-01-01,ZAMZ,4,2016.01,weekday_0,11320,10,401.00,off
2,2016,1,1,0,2016-01-01,ZIM,4,2016.01,weekday_0,5713,-170,-6817.00,off
3,2016,1,1,1,2016-01-01,ZAMC,4,2016.01,weekday_1,5714,160,6412.80,off
4,2016,1,1,1,2016-01-01,ZAMZ,4,2016.01,weekday_1,11322,10,400.80,off
...,...,...,...,...,...,...,...,...,...,...,...,...,...
221438,2021,3,31,23,2021-03-31,MOZ,2,2021.03,weekday_23,666292,164,6061.44,off
221439,2021,3,31,23,2021-03-31,NAM,2,2021.03,weekday_23,333309,42,1552.32,off
221440,2021,3,31,23,2021-03-31,RSA,2,2021.03,weekday_23,1665710,-150,-5544.00,off
221441,2021,3,31,23,2021-03-31,ZAMC,2,2021.03,weekday_23,333231,41,1515.36,off


In [201]:
flow_monthly=flows.groupby(['month_no','year','month','hour', 'hour_ref','node','cat']).sum().reset_index(['hour','month','year'])
flow_monthly.rename(columns={'flow': 'flow_sum'}, inplace=True)
flow_monthly=flow_monthly.reset_index().drop(['hour_ref','day_of_week','index'],axis=1)
flow_monthly

,month_no,node,cat,year,month,hour,flow_sum,value
0,2016.01,BOT,off,2016,1,0,-71,-3460.97
1,2016.01,MOZ,off,2016,1,0,1319,65858.37
2,2016.01,NAM,off,2016,1,0,50,2409.80
3,2016.01,RSA,off,2016,1,0,-104,-5100.52
4,2016.01,ZAMC,off,2016,1,0,1391,65631.17
...,...,...,...,...,...,...,...,...
27483,2021.03,RSA,standard,2021,3,23,-146,-5842.92
27484,2021.03,ZAMC,off,2021,3,23,478,20808.13
27485,2021.03,ZAMC,standard,2021,3,23,57,2281.14
27486,2021.03,ZIM,off,2021,3,23,-822,-35965.48


In [200]:
flow_monthly.to_csv('dam_flow_monthly.csv')

In [ ]:
flowa=flow.copy().rename(columns={'to':'from','from':'to'})
flow.loc[
    (
        (flow['from']=='RSAN') & (flow['to']=='MOZS') |
        (flow['from']=='MOZS') & (flow['to']=='RSAN') |
        (flow['from']=='MOZN_EDM') & (flow['to']=='ZIMA') |
        (flow['from']=='ZIMA') & (flow['to']=='MOZN_EDM') |
        (flow['from']=='MOZN_HCB') & (flow['to']=='MOZN_EDM') |
        (flow['from']=='MOZN_EDM') & (flow['to']=='MOZN_HCB')
    ) 
    ,['to','from']]=flowa.loc[
    (
        (flow['from']=='RSAN') & (flow['to']=='MOZS') |
        (flow['from']=='MOZS') & (flow['to']=='RSAN') |
        (flow['from']=='MOZN_EDM') & (flow['to']=='ZIMA') |
        (flow['from']=='ZIMA') & (flow['to']=='MOZN_EDM') |
        (flow['from']=='MOZN_HCB') & (flow['to']=='MOZN_EDM') |
        (flow['from']=='MOZN_EDM') & (flow['to']=='MOZN_HCB')
    )     
    ,['to','from']]